# Understanding the modal shifting and travel delays under short-term transport disruption – a case study of London tube strike

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
from defined_methods import merge_csv, merge_csv_flow

# Data preparation

In [ ]:
# read the administrative relationship encoding file
admin_rel = pd.read_csv(
    '/Users/zonghe/Library/CloudStorage/OneDrive-UnitedNationsDevelopmentProgramme/Paper/Tube Strike Behaviour/oa_lsoa_msoa_la_trans.csv')
admin_rel

In [ ]:
# read the spatial administrative boundary files
oa = gpd.read_file(
    '/Users/zonghe/Library/CloudStorage/OneDrive-UnitedNationsDevelopmentProgramme/Paper/Tube Strike Behaviour/lon_oa.gpkg')
# lsoa = gpd.read_file(
#     '/Users/zonghe/Library/CloudStorage/OneDrive-UnitedNationsDevelopmentProgramme/Paper/Tube Strike Behaviour/lon_lsoa.gpkg')
msoa = gpd.read_file(
    '/Users/zonghe/Library/CloudStorage/OneDrive-UnitedNationsDevelopmentProgramme/Paper/Tube Strike Behaviour/lon_msoa.gpkg')
msoa.plot()
plt.axis('off')


In [ ]:
msoa

## Individual

In [ ]:
device = merge_csv(
    '/Users/zonghe/Library/CloudStorage/OneDrive-UnitedNationsDevelopmentProgramme/Paper/Tube Strike Behaviour/travel_info_oa/device')
stay_time = merge_csv(
    '/Users/zonghe/Library/CloudStorage/OneDrive-UnitedNationsDevelopmentProgramme/Paper/Tube Strike Behaviour/travel_info_oa/stay time')
travel_time = merge_csv(
    '/Users/zonghe/Library/CloudStorage/OneDrive-UnitedNationsDevelopmentProgramme/Paper/Tube Strike Behaviour/travel_info_oa/travel time')
# drop the na values in the stay time dataset
stay_time = stay_time.dropna()
# convert the stay time from seconds to hours
stay_time['stay_time'] = stay_time['stay_time'].astype(int) / 3600


In [ ]:
# Filtering for March 1, 2022, and March 3, 2022

device_313 = device[(device['date'] == '02/22/22') | (device['date'] == '03/01/22') | (device['date'] == '03/03/22')].copy().reset_index(drop=True)
device_313['date'] = pd.to_datetime(device_313['date'], format='%m/%d/%y').dt.strftime('%m/%d/%y')

stay_time_313 = stay_time[(stay_time['date'] == '02/22/22') | (stay_time['date'] == '03/01/22') | (stay_time['date'] == '03/03/22')].copy().reset_index(
    drop=True)
stay_time_313['date'] = pd.to_datetime(stay_time_313['date'], format='%m/%d/%y').dt.strftime('%m/%d/%y')

travel_time_313 = travel_time[
    (travel_time['date'] == '02/22/22') | (travel_time['date'] == '03/01/22') | (travel_time['date'] == '03/03/22')].copy().reset_index(drop=True)
travel_time_313['date'] = pd.to_datetime(travel_time_313['date'], format='%m/%d/%y').dt.strftime('%m/%d/%y')

In [ ]:
# Device related behavioral test
test_device_lable = device_313['startid'].unique()[1]
# print(test_device_lable)
test_device = device_313[device_313['startid'] == test_device_lable]
# print(test_device)
test_device_stay = stay_time_313[stay_time_313['startid'] == test_device_lable]
# print(test_device_stay)
test_device_modes = travel_time_313[travel_time_313['startid'] == test_device_lable]
# print(test_device_travel)
# merge the test device status, stay time and travel time
test_device_status = pd.merge(test_device, test_device_stay, on=['startid', 'date', 'oa21cd', 'device_type'],
                              how='left')
test_device = test_device.drop_duplicates().reset_index(drop=True)
test_device_status

In [ ]:
test_device_modes

### Device behaviour calculation

In [ ]:
device_313_status = pd.merge(device_313, stay_time_313, on=['startid', 'date', 'oa21cd', 'device_type'], how='left')
device_313_status = device_313_status.drop_duplicates().reset_index(drop=True)
# insert the MSOA and LSOA column from 'admin_rel'
device_313_status = pd.merge(device_313_status, admin_rel[['oa21cd', 'msoa21cd','lsoa21cd']], on=['oa21cd'], how='left')
device_313_status = device_313_status.drop_duplicates().reset_index(drop=True)

device_313_modes = travel_time_313.groupby(['startid', 'date', 'update_mode']).agg(
    {'time': 'sum', 'distance': 'sum'}).reset_index()

In [ ]:
device_313_status

In [ ]:
device_313_modes

## System

In [ ]:
traffic_flows = merge_csv_flow(
    '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Zonghe Ma-STA-Investigating the relationship between modal shifting and road transport resilience under Tube strikes/Raw data/[XH]Traffic flow')
road_network = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Zonghe Ma-STA-Investigating the relationship between modal shifting and road transport resilience under Tube strikes/Raw data/[XH]road_network/road_network.shp'

# clean the traffic flow data
traffic_flows = traffic_flows.drop_duplicates(['toid', 'date'])
traffic_flows = traffic_flows.groupby(['toid', 'date']).agg(
    {'bus': 'sum', 'car': 'sum', 'cycle': 'sum', 'walks': 'sum', 'stationary': 'sum'}).reset_index()

lsoa = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Zonghe Ma-STA-Investigating the relationship between modal shifting and road transport resilience under Tube strikes/Raw data/London administrative boundaries/london_LSOA/london_LSOA.shp'

inoutter = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Zonghe Ma-STA-Investigating the relationship between modal shifting and road transport resilience under Tube strikes/Raw data/London administrative boundaries/lp-consultation-oct-2009-inner-outer-london-shp/lp-consultation-oct-2009-inner-outer-london.shp'
# tube_line = 'https://raw.githubusercontent.com/oobrien/vis/master/tubecreature/data/tfl_lines.json'
# tube_station = 'https://raw.githubusercontent.com/oobrien/vis/master/tubecreature/data/tfl_stations.json'

inoutter = gpd.read_file(inoutter)
inoutter.to_crs(epsg=27700, inplace=True)

lsoa = gpd.read_file(lsoa, crs={'init': 'epsg:27700'})
road_network = gpd.read_file(road_network, crs={'init': 'epsg:27700'})
road_network.rename(columns={'NAME': 'boroughs'}, inplace=True)
road_network.loc[:, ['cycle_lane', 'bus_lane']] = road_network[['cycle_lane', 'bus_lane']].fillna('n')

# clean the traffic flow data
traffic_flows = traffic_flows.drop_duplicates(['toid', 'date'])
traffic_flows = traffic_flows.groupby(['toid', 'date']).agg(
    {'bus': 'sum', 'car': 'sum', 'cycle': 'sum', 'walks': 'sum', 'stationary': 'sum'}).reset_index()
traffic_flows['total'] = traffic_flows['bus'] + traffic_flows['car'] + traffic_flows['cycle'] + traffic_flows[
    'walks'] + traffic_flows['stationary']

flows = pd.merge(
    road_network[
        ['toid', 'roadclassi', 'geometry', 'cycle_lane', 'bus_lane', 'boroughs', 'directiona', 'length', 'roadwidtha',
         'elevationg']],
    traffic_flows, left_on='toid', right_on='toid', how='left')
flows.set_geometry('geometry', inplace=True)

flows['classification'] = flows['roadclassi'].replace(
    {'Unknown': 'Local Road', 'Not Classified': 'Local Road', 'Unclassified': 'Local Road',
     'Classified Unnumbered': 'Local Road', 'A Road': 'Strategic Road', 'B Road': 'Strategic Road'})

flows.drop(columns=['roadclassi'], inplace=True)

stage_date = ['03/01/22', '02/22/22', '03/08/22']
flows = flows.loc[flows['date'].isin(stage_date)]

# label the regional level
flows = gpd.sjoin(flows, inoutter, how='inner', predicate='within')
flows = flows.drop(columns=['index_right', 'Source', 'Area_Ha', 'Shape_Leng', 'Shape_Area'])
flows.reset_index(drop=True, inplace=True)

regression = flows

# convert the dataframe
flows = pd.melt(flows,
                id_vars=['toid', 'classification', 'geometry', 'date', 'Boundary', 'cycle_lane', 'bus_lane',
                         'boroughs', 'directiona', 'length', 'roadwidtha', 'elevationg'],
                var_name='mode', value_name='flow')

flows = pd.pivot_table(flows,
                       index=['toid', 'classification', 'geometry', 'Boundary', 'mode', 'cycle_lane', 'bus_lane',
                              'boroughs', 'directiona', 'length', 'roadwidtha', 'elevationg'],
                       columns='date',
                       values='flow',
                       aggfunc='first').reset_index()

flows = flows.groupby(
    ['toid', 'mode', 'classification', 'geometry', 'Boundary', 'cycle_lane', 'bus_lane', 'boroughs', 'directiona',
     'length', 'roadwidtha', 'elevationg'],
    as_index=False).agg(
    {'03/01/22': 'first', '02/22/22': 'first', '03/08/22': 'first'})
# Calculate the impact and recovery flows for one strike
flows['impact_flow'] = flows['03/01/22'] - flows['02/22/22']
flows['recovery_flow'] = flows['03/08/22'] - flows['03/01/22']

# Calculate impact rate while avoiding division by zero
flows['impact_rate'] = flows.apply(
    lambda row: round(row['impact_flow'] / row['02/22/22'], 4) if row['02/22/22'] != 0 else 0, axis=1)
# Calculate recovery rate while avoiding division by zero
flows['recovery_rate'] = flows.apply(
    lambda row: round(row['recovery_flow'] / row['03/01/22'], 4) if row['03/01/22'] != 0 else 0, axis=1)

speed = merge_csv_flow(
    '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Zonghe Ma-STA-Investigating the relationship between modal shifting and road transport resilience under Tube strikes/Raw data/Traffic flow with speed')
speed = speed.groupby(['travel_mode', 'toid', 'date']).agg({'speed_overall': 'mean'}).reset_index()
speed = speed.pivot(index=['toid', 'travel_mode'], columns='date', values='speed_overall').reset_index()
speed.rename(columns={'03/01/22': 'speed_03/01/22', '02/22/22': 'speed_02/22/22', '03/08/22': 'speed_03/08/22'},
             inplace=True)

columns_to_sum = ['speed_03/01/22', 'speed_02/22/22', 'speed_03/08/22']
toid_mode_totals = speed.groupby(['toid', 'travel_mode'])[columns_to_sum].sum().reset_index()

# 创建一个包含 'total' 行的 DataFrame
total_row = toid_mode_totals.groupby('toid')[columns_to_sum].sum().reset_index()
total_row['travel_mode'] = 'total'

speed = pd.concat([toid_mode_totals, total_row], ignore_index=True)

# Calculate the impact and recovery flows for one strike
speed['impact_speed'] = speed['speed_03/01/22'] - speed['speed_02/22/22']
speed['recovery_speed'] = speed['speed_03/08/22'] - speed['speed_03/01/22']

speed['impact_speed_rate'] = speed.apply(
    lambda row: round(row['impact_speed'] / row['speed_02/22/22'], 4) if row['speed_02/22/22'] != 0 else 0, axis=1)
# Calculate recovery rate while avoiding division by zero
speed['recovery_speed_rate'] = speed.apply(
    lambda row: round(row['recovery_speed'] / row['speed_03/01/22'], 4) if row['speed_03/01/22'] != 0 else 0, axis=1)
flows = pd.merge(flows, speed, left_on=['toid', 'mode'], right_on=['toid', 'travel_mode'], how='left')
flows.drop(columns=['travel_mode'], inplace=True)

All = flows.copy()
All


# Indicators building

## Individual behaviours

### Trip distance & Trip time

#### per person

In [ ]:
# Group by 'date' and 'update_mode', and aggregate 'time' and 'distance' by sum
tripdistance_ppl = device_313_modes.groupby(['date','startid']).agg(time_sum=('time', 'sum'), distance_sum=('distance', 'sum')).reset_index()
tripdistance_ppl[['time_sum', 'distance_sum']] = tripdistance_ppl[['time_sum', 'distance_sum']].round(2)


# Calculating mean values and renaming columns
tripdistance_ppl['time_mean'] = device_313_modes.groupby(['date','startid'])['time'].transform('mean').round(2)
tripdistance_ppl['distance_mean'] = device_313_modes.groupby(['date','startid'])['distance'].transform('mean').round(2)

tripdistance_ppl

#### per mode

In [ ]:
# Group by 'date' and 'update_mode', and aggregate 'time' and 'distance' by sum
tripdistance_pm = device_313_modes.groupby(['date', 'update_mode']).agg(time_sum=('time', 'sum'), distance_sum=('distance', 'sum')).reset_index()
tripdistance_pm[['time_sum', 'distance_sum']] = tripdistance_pm[['time_sum', 'distance_sum']].round(2)


# Calculating mean values and renaming columns
tripdistance_pm['time_mean'] = device_313_modes.groupby(['date', 'update_mode'])['time'].transform('mean').round(2)
tripdistance_pm['distance_mean'] = device_313_modes.groupby(['date', 'update_mode'])['distance'].transform('mean').round(2)

tripdistance_pm

#### 1111111 per OA

In [ ]:
# merge dfs to aggrate in OA level
device_313_merged = pd.merge(device_313_status, device_313_modes, on=['date','startid'], how='left')

# Group by 'date' and 'oa21xd_x', and aggregate 'time' and 'distance' by sum
tripdistance_oa = device_313_merged.groupby(['date', 'oa21cd']).agg(time_sum=('time', 'sum'), distance_sum=('distance', 'sum')).reset_index()
tripdistance_oa[['time_sum', 'distance_sum']] = tripdistance_oa[['time_sum', 'distance_sum']].round(2)


# Calculating mean values and renaming columns
tripdistance_oa['time_mean'] = device_313_merged.groupby(['date', 'oa21cd'])['time'].transform('mean').round(2)
tripdistance_oa['distance_mean'] = device_313_merged.groupby(['date', 'oa21cd'])['distance'].transform('mean').round(2)

# calulations for device type
# Count the occurrences of each device type
device_type_counts = device_313_merged.groupby(['date', 'oa21cd', 'device_type']).size().reset_index(name='count')
# Calculate the total count for each 'date' and 'oa21cd' group
total_counts = device_type_counts.groupby(['date', 'oa21cd'])['count'].transform('sum')
# Calculate the rate (percentage) for each device type
device_type_counts['rate'] = (device_type_counts['count'] / total_counts) * 100

# Pivot the data
device_type_pivot = device_type_counts.pivot_table(index=['date', 'oa21cd'], columns='device_type', values='rate').reset_index()
# Reset the column names
device_type_pivot.columns.name = None
device_type_pivot.columns = ['date', 'oa21cd'] + [f'rate_{device_type}' for device_type in device_type_pivot.columns[2:]]
# Merge with tripdistance_oa
tripdistance_oa = pd.merge(tripdistance_oa, device_type_pivot, on=['date', 'oa21cd'], how='left').round(4)
tripdistance_oa['trip_rate'] = tripdistance_oa['rate_resident'] + tripdistance_oa['rate_passthrough']

tripdistance_oa = pd.merge(tripdistance_oa, oa, on=['oa21cd'], how='left')
tripdistance_oa = gpd.GeoDataFrame(tripdistance_oa, crs={'init': 'epsg:27700'}, geometry='geometry')

tripdistance_oa

In [ ]:
tripdistance_oa.plot(column='time_sum', cmap='coolwarm', legend=True, scheme= 'quantiles', k=5)
plt.axis('off')

### Trip rate

#### per person

In [ ]:
# the rate of trip counts: selecte the non-residents status rates from the device status dataset

# Group by 'date' and then calculate the normalized value counts for each 'device_type'
device_type_rate_ppl = device_313_status.groupby(['date','startid'])['device_type'].value_counts(normalize=True).unstack(fill_value=0)

# Calculate the proportion of non-resident devices for each person by each date
triprate_ppl = 1 - device_type_rate_ppl.get('resident', 0)

triprate_ppl = pd.DataFrame(triprate_ppl).reset_index()
triprate_ppl.rename({'resident': 'trip_rate'}, axis=1, inplace=True)

triprate_ppl

#### per mode

In [ ]:
triprate_mode = device_313_merged.groupby(['date','startid'])['device_type'].value_counts(normalize=True).unstack(fill_value=0)

#### per OA

In [ ]:
triprate_oa =tripdistance_oa[['date', 'oa21cd', 'trip_rate','geometry']].copy()
triprate_oa

In [ ]:
tripdistance_oa.plot(column='trip_rate', cmap='coolwarm', legend=True, scheme= 'quantiles', k=5)
plt.axis('off')

### Travel time & distance

In [ ]:
# Group by 'date' and 'update_mode', and aggregate 'time' and 'distance' by sum
travel = device_313_modes.groupby(['date']).agg(time_sum=('time', 'sum'), distance_sum=('distance', 'sum')).reset_index()
travel[['time_sum', 'distance_sum']] = travel[['time_sum', 'distance_sum']].round(2)


# Calculating mean values and renaming columns
travel['time_mean'] = device_313_modes.groupby(['date','startid'])['time'].transform('mean').round(2)
travel['distance_mean'] = device_313_modes.groupby(['date','startid'])['distance'].transform('mean').round(2)

travel

### Travel rate

In [ ]:
# the rate of trip counts: selecte the non-residents status rates from the device status dataset

# Group by 'date' and then calculate the normalized value counts for each 'device_type'
device_type_rate_pd = device_313_status.groupby('date')['device_type'].value_counts(normalize=True).unstack(fill_value=0)

# Calculate the proportion of non-resident devices for each date
travelrate = 1 - device_type_rate_pd.get('resident', 0)

travelrate


## System performance

### Flow change

### Traffic congestion